In [1]:
# Backbone models.
# !pip install git+https://github.com/rcmalli/keras-vggface.git
# !pip install keras_vggface
# !pip install keras_applications

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from utils.models import SiameseNetwork, BackboneNetwork
from utils.funcs import *

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

2021-08-15 16:33:51.405345: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Using VGGFace compatible with TensorFlow2.


In [3]:
# Hyperparameters
BASE_MODEL = 'vgg16'
INPUT_SHAPE = (224, 224,)
FINE_TUNE = True
EPOCHS = 15
DIST = 'default'
LEARNING_RATE = 0.00003
PATIENCE = 5
BATCH_SIZE = 16

# Modify paths as per your method of saving them
BASE_PATH = "/root/KinshipRecognition"
TRAIN_FILE_PATH = f"{BASE_PATH}/data/aug_train_ds.csv"
TRAIN_FOLDERS_PATH = f"{BASE_PATH}/data/train/train-faces/"

# Output file
MODEL_NAME = f"ensemble_{BASE_MODEL}_{DIST}_finetune{FINE_TUNE}"

# All images belonging to families F0X** will be used to create the validation set while training the model
val_families_list = ["F00", "F01", "F02", "F03", "F04", "F05", "F06", "F07", "F08", "F09"]

In [4]:
for i in range(len(val_families_list)):

    print('##############################')
    print(f'Iteration {i}: Validation on {val_families_list[i]}')
    print('##############################')

    train, val, train_person_to_images_map, val_person_to_images_map = get_train_val(val_families_list[i], 
                                                                                    TRAIN_FOLDERS_PATH,
                                                                                    TRAIN_FILE_PATH)
    file_path = f"{BASE_PATH}/log/model/{MODEL_NAME}_{i}.h5"
    checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.33, patience=PATIENCE, verbose=1,
                                         min_lr=0.00002)
    callbacks_list = [checkpoint, reduce_on_plateau]
    
    model = SiameseNetwork(BASE_MODEL, fine_tune=FINE_TUNE, dist=DIST,optimizer=Adam(LEARNING_RATE)).build()
    
    history = model.fit(gen(train, train_person_to_images_map, INPUT_SHAPE, TRAIN_FOLDERS_PATH, batch_size=16), 
                        validation_data=gen(val, val_person_to_images_map, INPUT_SHAPE, TRAIN_FOLDERS_PATH, batch_size=16), 
                        epochs=EPOCHS, steps_per_epoch=300, validation_steps=200,
                        verbose=1, callbacks=callbacks_list, 
                        use_multiprocessing=False, workers=1)

##############################
Iteration 0: Validation on F00
##############################


2021-08-15 16:34:15.669924: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-15 16:34:15.671328: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-15 16:34:15.790286: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-08-15 16:34:15.790319: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-08-15 16:34:15.791615: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-08-15 16:34:15.791673: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

input_3 False
conv1_1 False
conv1_2 False
pool1 False
conv2_1 False
conv2_2 False
pool2 False
conv3_1 False
conv3_2 False
conv3_3 False
pool3 False
conv4_1 False
conv4_2 False
conv4_3 False
pool4 False
conv5_1 False
conv5_2 False
conv5_3 True
pool5 True
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_vgg16 (Functional)      (None, None, None, 5 14714688    input_1[0][0]                    
                                      

2021-08-15 16:34:17.280778: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-15 16:34:17.298976: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2300000000 Hz


Epoch 1/15


...

Epoch 1/15
300/300 [==============================] - 50s 163ms/step - loss: 0.7652 - acc: 0.5754 - val_loss: 0.6966 - val_acc: 0.6100

Epoch 00001: val_acc improved from -inf to 0.61000, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_5.h5
Epoch 2/15
300/300 [==============================] - 47s 157ms/step - loss: 0.5588 - acc: 0.7330 - val_loss: 0.6497 - val_acc: 0.6562

Epoch 00002: val_acc improved from 0.61000 to 0.65625, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_5.h5
Epoch 3/15
300/300 [==============================] - 47s 157ms/step - loss: 0.4834 - acc: 0.7815 - val_loss: 0.6630 - val_acc: 0.6561

Epoch 00003: val_acc did not improve from 0.65625
Epoch 4/15
300/300 [==============================] - 47s 157ms/step - loss: 0.4389 - acc: 0.8110 - val_loss: 0.6597 - val_acc: 0.6714

Epoch 00004: val_acc improved from 0.65625 to 0.67141, saving model to /root/KinshipRecognition/log/model/ensemble_

Epoch 1/15
300/300 [==============================] - 53s 175ms/step - loss: 0.7237 - acc: 0.5793 - val_loss: 0.6365 - val_acc: 0.6603

Epoch 00001: val_acc improved from -inf to 0.66031, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_6.h5
Epoch 2/15
300/300 [==============================] - 47s 156ms/step - loss: 0.5379 - acc: 0.7525 - val_loss: 0.6015 - val_acc: 0.6817

Epoch 00002: val_acc improved from 0.66031 to 0.68172, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_6.h5
Epoch 3/15
300/300 [==============================] - 47s 158ms/step - loss: 0.4718 - acc: 0.7925 - val_loss: 0.6183 - val_acc: 0.6897

Epoch 00003: val_acc improved from 0.68172 to 0.68969, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_6.h5
Epoch 4/15
300/300 [==============================] - 46s 154ms/step - loss: 0.4371 - acc: 0.8052 - val_loss: 0.6029 - val_acc: 0.7086

Epoch 00004: val_a

Epoch 1/15
300/300 [==============================] - 49s 161ms/step - loss: 0.7052 - acc: 0.5725 - val_loss: 0.6980 - val_acc: 0.6098

Epoch 00001: val_acc improved from -inf to 0.60984, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_7.h5
Epoch 2/15
300/300 [==============================] - 111s 372ms/step - loss: 0.5508 - acc: 0.7400 - val_loss: 0.6154 - val_acc: 0.6797

Epoch 00002: val_acc improved from 0.60984 to 0.67969, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_7.h5
Epoch 3/15
300/300 [==============================] - 251s 839ms/step - loss: 0.4830 - acc: 0.7842 - val_loss: 0.6194 - val_acc: 0.6784

Epoch 00003: val_acc did not improve from 0.67969
Epoch 4/15
300/300 [==============================] - 54s 179ms/step - loss: 0.4377 - acc: 0.8052 - val_loss: 0.7362 - val_acc: 0.6145

Epoch 00004: val_acc did not improve from 0.67969
Epoch 5/15
300/300 [==============================] - 104s 347m

Epoch 1/15
300/300 [==============================] - 56s 182ms/step - loss: 0.7191 - acc: 0.5788 - val_loss: 0.5952 - val_acc: 0.7047

Epoch 00001: val_acc improved from -inf to 0.70469, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_8.h5
Epoch 2/15
300/300 [==============================] - 54s 180ms/step - loss: 0.5653 - acc: 0.7232 - val_loss: 0.5440 - val_acc: 0.7359

Epoch 00002: val_acc improved from 0.70469 to 0.73594, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_8.h5
Epoch 3/15
300/300 [==============================] - 54s 181ms/step - loss: 0.5008 - acc: 0.7717 - val_loss: 0.5480 - val_acc: 0.7247

Epoch 00003: val_acc did not improve from 0.73594
Epoch 4/15
300/300 [==============================] - 54s 182ms/step - loss: 0.4388 - acc: 0.8118 - val_loss: 0.5503 - val_acc: 0.7353

Epoch 00004: val_acc did not improve from 0.73594
Epoch 5/15
300/300 [==============================] - 55s 184ms/s

Epoch 1/15
300/300 [==============================] - 162s 515ms/step - loss: 0.7236 - acc: 0.5562 - val_loss: 0.6370 - val_acc: 0.6517

Epoch 00001: val_acc improved from -inf to 0.65172, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_9.h5
Epoch 2/15
300/300 [==============================] - 57s 190ms/step - loss: 0.5595 - acc: 0.7283 - val_loss: 0.6296 - val_acc: 0.6755

Epoch 00002: val_acc improved from 0.65172 to 0.67547, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_9.h5
Epoch 3/15
300/300 [==============================] - 57s 190ms/step - loss: 0.4792 - acc: 0.7863 - val_loss: 0.6441 - val_acc: 0.6866

Epoch 00003: val_acc improved from 0.67547 to 0.68656, saving model to /root/KinshipRecognition/log/model/ensemble_vgg16_default_finetuneTrue_9.h5
Epoch 4/15
300/300 [==============================] - 56s 187ms/step - loss: 0.4560 - acc: 0.7928 - val_loss: 0.6191 - val_acc: 0.7047

Epoch 00004: val_

In [5]:
test_path = f"{BASE_PATH}/data/test/"
submission = pd.read_csv(f'{BASE_PATH}/data/test_ds.csv')
preds_for_sub = np.zeros(submission.shape[0])
all_preds = list()
for i in range(len(val_families_list)):

    print('##############################')
    print(f'Iteration {i}: Validation on {val_families_list[i]}')
    print('##############################')
    
    model = SiameseNetwork(BASE_MODEL, fine_tune=FINE_TUNE, dist=DIST,optimizer=Adam(LEARNING_RATE)).build()
    file_path = f"{BASE_PATH}/log/model/{MODEL_NAME}_{i}.h5"
    model.load_weights(file_path)

    # Predictions
    step = BATCH_SIZE * 2
    predictions = []
    for j in range(0, len(submission.p1.values), step):
        X1 = submission.p1.values[j:j+step]
        X1 = np.array([read_img(test_path + x, INPUT_SHAPE) for x in X1])
        X2 = submission.p2.values[j:j+step]
        X2 = np.array([read_img(test_path + x, INPUT_SHAPE) for x in X2])
        if len(X1) < step:
            diff = step - len(X1)
            aux_X = submission.p1.values[:diff]
            aux_X = np.array([read_img(test_path + x, INPUT_SHAPE) for x in aux_X])
            aux_X1 = np.r_[X1, aux_X]
            aux_X2 = np.r_[X2, aux_X]
            pred = model.predict([aux_X1, aux_X2]).ravel().tolist()
            pred = pred[:len(X1)]
        else:
            pred = model.predict([X1, X2]).ravel().tolist()
        predictions += pred        

    all_preds.append(np.array(predictions))
    preds_for_sub += np.array(predictions) / len(val_families_list)

    
all_preds = np.asarray(all_preds).T
submission['score'] = preds_for_sub
pd.DataFrame(all_preds).to_csv(f"{BASE_PATH}/log/results/{MODEL_NAME}_allpreds.csv", index=False)
submission.to_csv(f"{BASE_PATH}/log/results/{MODEL_NAME}.csv", index=False)

##############################
Iteration 0: Validation on F00
##############################
input_33 False
conv1_1 False
conv1_2 False
pool1 False
conv2_1 False
conv2_2 False
pool2 False
conv3_1 False
conv3_2 False
conv3_3 False
pool3 False
conv4_1 False
conv4_2 False
conv4_3 False
pool4 False
conv5_1 False
conv5_2 False
conv5_3 True
pool5 True
Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_vgg16 (Functional)      (None, N

##############################
Iteration 2: Validation on F02
##############################
input_39 False
conv1_1 False
conv1_2 False
pool1 False
conv2_1 False
conv2_2 False
pool2 False
conv3_1 False
conv3_2 False
conv3_3 False
pool3 False
conv4_1 False
conv4_2 False
conv4_3 False
pool4 False
conv5_1 False
conv5_2 False
conv5_3 True
pool5 True
Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_vgg16 (Functional)      (None, N

##############################
Iteration 4: Validation on F04
##############################
input_45 False
conv1_1 False
conv1_2 False
pool1 False
conv2_1 False
conv2_2 False
pool2 False
conv3_1 False
conv3_2 False
conv3_3 False
pool3 False
conv4_1 False
conv4_2 False
conv4_3 False
pool4 False
conv5_1 False
conv5_2 False
conv5_3 True
pool5 True
Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_43 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_44 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_vgg16 (Functional)      (None, N

##############################
Iteration 6: Validation on F06
##############################
input_51 False
conv1_1 False
conv1_2 False
pool1 False
conv2_1 False
conv2_2 False
pool2 False
conv3_1 False
conv3_2 False
conv3_3 False
pool3 False
conv4_1 False
conv4_2 False
conv4_3 False
pool4 False
conv5_1 False
conv5_2 False
conv5_3 True
pool5 True
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_49 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_50 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_vgg16 (Functional)      (None, N

##############################
Iteration 8: Validation on F08
##############################
input_57 False
conv1_1 False
conv1_2 False
pool1 False
conv2_1 False
conv2_2 False
pool2 False
conv3_1 False
conv3_2 False
conv3_3 False
pool3 False
conv4_1 False
conv4_2 False
conv4_3 False
pool4 False
conv5_1 False
conv5_2 False
conv5_3 True
pool5 True
Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_55 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_56 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_vgg16 (Functional)      (None, N

In [6]:
print(np.sum(preds_for_sub <= 0.5))
print(len(preds_for_sub), '\n')
for line in preds_for_sub:
    print(line)

1798
3000 

0.2441397536545992
0.056035458017140626
0.6418508321046829
0.8277451455593109
0.3316828101873398
0.187518893647939
0.4885197073221207
0.051225732732564214
0.8306846559047699
0.21835185950621966
0.09915623171254992
0.07654145467095076
0.7290391743183138
0.6528122067451477
0.0479622072307393
0.7597625464200972
0.23195278644561765
0.3822233133018017
0.1327974282205105
0.16195716969668866
0.6589959889650344
0.21032285541296006
0.3245303988456726
0.11561125516891477
0.6461426764726639
0.17463964652270078
0.10817232290282845
0.31765098385512824
0.28670673542656
0.4687825370579957
0.7079221695661545
0.3499152407050133
0.41689118370413786
0.2039002156816423
0.21337272226810455
0.10151751330122351
0.49420951791107653
0.5998030148446559
0.7320288002490998
0.06408761695493013
0.220848099142313
0.07428454058244824
0.5238889738917352
0.6701879173517227
0.5900618851184846
0.28428158089518546
0.7478068739175796
0.11228138450533151
0.8670121371746065
0.1911881674081087
0.24483842253684998
